# データサイエンス100本ノック（構造化データ加工編） - SQL

## はじめに
- データベースはPostgreSQL13です
- 初めに以下のセルを実行してください
- セルに %%sql と記載することでSQLを発行することができます
- jupyterからはdescribeコマンドによるテーブル構造の確認ができないため、テーブル構造を確認する場合はlimitを指定したSELECTなどで代用してください
- 使い慣れたSQLクライアントを使っても問題ありません（接続情報は以下の通り）
  - IPアドレス：Docker Desktopの場合はlocalhost、Docker toolboxの場合は192.168.99.100
  - Port:5432
  - database名: dsdojo_db
  - ユーザ名：padawan
  - パスワード:padawan12345
- 大量出力を行うとJupyterが固まることがあるため、出力件数は制限することを推奨します（設問にも出力件数を記載）
    - 結果確認のために表示させる量を適切にコントロールし、作業を軽快にすすめる技術もデータ加工には求められます
- 大量結果が出力された場合は、ファイルが重くなり以降開けなくなることもあります
    - その場合、作業結果は消えますがファイルをGitHubから取り直してください
    - vimエディタなどで大量出力範囲を削除することもできます
- 名前、住所等はダミーデータであり、実在するものではありません

In [2]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

# 使い方
- セルの先頭に%%sqlと記載し、２行目以降にSQLを記述することでJupyterからPostgreSQLに対しSQLを実行できます。

In [2]:
%%sql
SELECT 'このように実行できます' AS sample;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


sample
このように実行できます


# データ加工100本ノック

---
> S-001: レシート明細データ（receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [2]:
%%sql

SELECT
    *
FROM
    receipt
LIMIT
    10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158
20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81
20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170
20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25
20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90
20190605,1559692800,S13003,1112,1,CS003515000195,P050102002,1,138
20181205,1543968000,S14024,1102,2,CS024514000042,P080101005,1,30
20190922,1569110400,S14040,1102,1,CS040415000178,P070501004,1,128
20170504,1493856000,S13020,1112,2,ZZ000000000000,P071302010,1,770
20191010,1570665600,S14027,1102,1,CS027514000015,P071101003,1,680


---
> S-002: レシート明細データ（receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [5]:
%%sql

SELECT 
    sales_ymd, 
    customer_id, 
    product_cd, 
    amount
FROM 
    receipt
LIMIT 
    10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


sales_ymd,customer_id,product_cd,amount
20181103,CS006214000001,P070305012,158
20181118,CS008415000097,P070701017,81
20170712,CS028414000014,P060101005,170
20190205,ZZ000000000000,P050301001,25
20180821,CS025415000050,P060102007,90
20190605,CS003515000195,P050102002,138
20181205,CS024514000042,P080101005,30
20190922,CS040415000178,P070501004,128
20170504,ZZ000000000000,P071302010,770
20191010,CS027514000015,P071101003,680


---
> S-003: レシート明細データ（receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更しながら抽出すること。

In [10]:
%%sql

SELECT
    sales_ymd AS sales_date,
    customer_id,
    product_cd,
    amount
FROM
    receipt
LIMIT
    5
;
   

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


sales_date,customer_id,product_cd,amount
20181103,CS006214000001,P070305012,158
20181118,CS008415000097,P070701017,81
20170712,CS028414000014,P060101005,170
20190205,ZZ000000000000,P050301001,25
20180821,CS025415000050,P060102007,90


---
> S-004: レシート明細データ（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [9]:
%%sql

SELECT
    sales_ymd,
    customer_id,
    product_cd,
    amount
FROM
    receipt
WHERE
    customer_id = 'CS018205000001'
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


sales_ymd,customer_id,product_cd,amount
20180911,CS018205000001,P071401012,2200
20180414,CS018205000001,P060104007,600
20170614,CS018205000001,P050206001,990
20170614,CS018205000001,P060702015,108
20190216,CS018205000001,P071005024,102


---
> S-005: レシート明細データ（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [11]:
%%sql

SELECT 
    sales_ymd,
    customer_id,
    product_cd,
    amount
FROM
    receipt
WHERE
    customer_id = 'CS018205000001'
    AND amount >= 1000
LIMIT
    5
;


*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


sales_ymd,customer_id,product_cd,amount
20180911,CS018205000001,P071401012,2200
20190226,CS018205000001,P071401020,2200
20180911,CS018205000001,P071401005,1100


---
> S-006: レシート明細データ（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [13]:
%%sql

SELECT
    sales_ymd,
    customer_id,
    product_cd,
    quantity,
    amount
FROM
    receipt
WHERE
    customer_id = 'CS018205000001'
    AND ( 
        amount >= 1000 
        OR quantity >= 5 
        )
LIMIT 
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


sales_ymd,customer_id,product_cd,quantity,amount
20180911,CS018205000001,P071401012,1,2200
20180414,CS018205000001,P060104007,6,600
20170614,CS018205000001,P050206001,5,990
20190226,CS018205000001,P071401020,1,2200
20180911,CS018205000001,P071401005,1,1100


---
> S-007: レシート明細データ（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [14]:
%%sql

SELECT 
    sales_ymd,
    customer_id,
    product_cd,
    amount
FROM
    receipt
WHERE
    customer_id = 'CS018205000001'
    AND amount BETWEEN 1000 AND 2000
LIMIT
5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


sales_ymd,customer_id,product_cd,amount
20180911,CS018205000001,P071401005,1100


---
> S-008: レシート明細データ（receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [15]:
%%sql

SELECT
    sales_ymd,
    customer_id,
    product_cd,
    amount
FROM
    receipt
WHERE
    customer_id = 'CS018205000001'
    AND product_cd != 'P071401019'
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


sales_ymd,customer_id,product_cd,amount
20180911,CS018205000001,P071401012,2200
20180414,CS018205000001,P060104007,600
20170614,CS018205000001,P050206001,990
20170614,CS018205000001,P060702015,108
20190216,CS018205000001,P071005024,102


---
> S-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
>
> `SELECT * FROM store WHERE NOT (prefecture_cd = '13' OR floor_area > 900)`

In [16]:
%%sql

SELECT
    *
FROM
    store
WHERE
    prefecture_cd != '13'
    AND floor_area <= 900
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0
S12013,習志野店,12,千葉県,千葉県習志野市芝園一丁目,チバケンナラシノシシバゾノイッチョウメ,047-123-4002,140.022,35.66122,808.0


---
> S-010: 店舗データ（store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [20]:
%%sql

SELECT
    *
FROM
    store
WHERE
    store_cd LIKE 'S14%'
LIMIT
    10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0
S14012,本牧和田店,14,神奈川県,神奈川県横浜市中区本牧和田,カナガワケンヨコハマシナカクホンモクワダ,045-123-4034,139.6582,35.42156,1341.0
S14046,北山田店,14,神奈川県,神奈川県横浜市都筑区北山田一丁目,カナガワケンヨコハマシツヅキクキタヤマタイッチョウメ,045-123-4049,139.5916,35.56189,831.0
S14022,逗子店,14,神奈川県,神奈川県逗子市逗子一丁目,カナガワケンズシシズシイッチョウメ,046-123-4036,139.5789,35.29642,1838.0
S14011,日吉本町店,14,神奈川県,神奈川県横浜市港北区日吉本町四丁目,カナガワケンヨコハマシコウホククヒヨシホンチョウヨンチョウメ,045-123-4033,139.6316,35.54655,890.0


---
> S-011: 顧客データ（customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [21]:
%%sql

SELECT
    *
FROM
    customer
WHERE
    customer_id LIKE '%1'
LIMIT
    10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0
CS028811000001,堀井 かおり,1,女性,1933-03-27,86,245-0016,神奈川県横浜市泉区和泉町**********,S14028,20160115,0-00000000-0
CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
CS028314000011,小菅 あおい,1,女性,1983-11-26,35,246-0038,神奈川県横浜市瀬谷区宮沢**********,S14028,20151123,1-20080426-5
CS039212000051,藤島 恵梨香,1,女性,1997-02-03,22,166-0001,東京都杉並区阿佐谷北**********,S13039,20171121,1-20100215-4
CS015412000111,松居 奈月,1,女性,1972-10-04,46,136-0071,東京都江東区亀戸**********,S13015,20150629,0-00000000-0
CS004702000041,野島 洋,0,男性,1943-08-24,75,176-0022,東京都練馬区向山**********,S13004,20170218,0-00000000-0
CS041515000001,栗田 千夏,1,女性,1967-01-02,52,206-0001,東京都多摩市和田**********,S13041,20160422,E-20100803-F
CS029313000221,北条 ひかり,1,女性,1987-06-19,31,279-0011,千葉県浦安市美浜**********,S12029,20180810,0-00000000-0
CS034312000071,望月 奈央,1,女性,1980-09-20,38,213-0026,神奈川県川崎市高津区久末**********,S14034,20160106,0-00000000-0


---
> S-012: 店舗データ（store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [22]:
%%sql

SELECT
    *
FROM
    store
WHERE
    address LIKE '%横浜市%'
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
S14040,長津田店,14,神奈川県,神奈川県横浜市緑区長津田みなみ台五丁目,カナガワケンヨコハマシミドリクナガツタミナミダイゴチョウメ,045-123-4046,139.4994,35.52398,1548.0
S14050,阿久和西店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4053,139.4961,35.45918,1830.0
S14028,二ツ橋店,14,神奈川県,神奈川県横浜市瀬谷区二ツ橋町,カナガワケンヨコハマシセヤクフタツバシチョウ,045-123-4042,139.4963,35.46304,1574.0


---
> S-013: 顧客データ（customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [25]:
%%sql

SELECT
    *
FROM
    customer
WHERE
    status_cd ~ '^[A-F]'
LIMIT
    10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C
CS015414000103,奥野 陽子,1,女性,1977-08-09,41,136-0073,東京都江東区北砂**********,S13015,20150722,B-20100609-B
CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
CS029415000023,梅田 里穂,1,女性,1976-01-17,43,279-0043,千葉県浦安市富士見**********,S12029,20150610,D-20100918-E
CS035415000029,寺沢 真希,9,不明,1977-09-27,41,158-0096,東京都世田谷区玉川台**********,S13035,20141220,F-20101029-F
CS031415000106,宇野 由美子,1,女性,1970-02-26,49,151-0053,東京都渋谷区代々木**********,S13031,20150201,F-20100511-E
CS029215000025,石倉 美帆,1,女性,1993-09-28,25,279-0022,千葉県浦安市今川**********,S12029,20150708,B-20100820-C
CS033605000005,猪股 雄太,0,男性,1955-12-05,63,246-0031,神奈川県横浜市瀬谷区瀬谷**********,S14033,20150425,F-20100917-E
CS033415000229,板垣 菜々美,1,女性,1977-11-07,41,246-0021,神奈川県横浜市瀬谷区二ツ橋町**********,S14033,20150712,F-20100326-E
CS008415000145,黒谷 麻緒,1,女性,1977-06-27,41,157-0067,東京都世田谷区喜多見**********,S13008,20150829,F-20100622-F


---
> S-014: 顧客データ（customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [26]:
%%sql

SELECT
    *
FROM
    customer
WHERE
    status_cd ~ '[1-9]$'
LIMIT
    10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS001215000145,田崎 美紀,1,女性,1995-03-29,24,144-0055,東京都大田区仲六郷**********,S13001,20170605,6-20090929-2
CS033513000180,安斎 遥,1,女性,1962-07-11,56,241-0823,神奈川県横浜市旭区善部町**********,S14033,20150728,6-20080506-5
CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
CS040412000191,川井 郁恵,1,女性,1977-01-05,42,226-0021,神奈川県横浜市緑区北八朔町**********,S14040,20151101,1-20091025-4
CS009315000023,皆川 文世,1,女性,1980-04-15,38,154-0012,東京都世田谷区駒沢**********,S13009,20150319,5-20080322-1
CS015315000033,福士 璃奈子,1,女性,1983-03-17,36,135-0043,東京都江東区塩浜**********,S13015,20141024,4-20080219-3
CS023513000066,神戸 そら,1,女性,1961-12-17,57,210-0005,神奈川県川崎市川崎区東田町**********,S14023,20150915,5-20100524-9
CS035513000134,市川 美帆,1,女性,1960-03-27,59,156-0053,東京都世田谷区桜**********,S13035,20150227,8-20100711-9
CS001515000263,高松 夏空,1,女性,1962-11-09,56,144-0051,東京都大田区西蒲田**********,S13001,20160812,1-20100804-1
CS040314000027,鶴田 きみまろ,9,不明,1986-03-26,33,226-0027,神奈川県横浜市緑区長津田**********,S14040,20150122,2-20080426-4


---
> S-015: 顧客データ（customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [27]:
%%sql

SELECT
    *
FROM
    customer
WHERE
    status_cd ~ '^[A-F].*[1-9]$'
LIMIT
    10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS011215000048,芦田 沙耶,1,女性,1992-02-01,27,223-0062,神奈川県横浜市港北区日吉本町**********,S14011,20150228,C-20100421-9
CS022513000105,島村 貴美子,1,女性,1962-03-12,57,249-0002,神奈川県逗子市山の根**********,S14022,20150320,A-20091115-7
CS001515000096,水野 陽子,9,不明,1960-11-29,58,144-0053,東京都大田区蒲田本町**********,S13001,20150614,A-20100724-7
CS013615000053,西脇 季衣,1,女性,1953-10-18,65,261-0026,千葉県千葉市美浜区幕張西**********,S12013,20150128,B-20100329-6
CS020412000161,小宮 薫,1,女性,1974-05-21,44,174-0042,東京都板橋区東坂下**********,S13020,20150822,B-20081021-3
CS001215000097,竹中 あさみ,1,女性,1990-07-25,28,146-0095,東京都大田区多摩川**********,S13001,20170315,A-20100211-2
CS035212000007,内村 恵梨香,1,女性,1990-12-04,28,152-0023,東京都目黒区八雲**********,S13035,20151013,B-20101018-6
CS002515000386,野田 コウ,1,女性,1963-05-30,55,185-0013,東京都国分寺市西恋ケ窪**********,S13002,20160410,C-20100127-8
CS001615000372,稲垣 寿々花,1,女性,1956-10-29,62,144-0035,東京都大田区南蒲田**********,S13001,20170403,A-20100104-1
CS032512000121,松井 知世,1,女性,1962-09-04,56,210-0011,神奈川県川崎市川崎区富士見**********,S13032,20150727,A-20100103-5


---
> S-016: 店舗データ（store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [30]:
%%sql

SELECT
    *
FROM
    store
WHERE
    tel_no ~ '^\d{3}-\d{3}-\d{4}$'
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,store_name,prefecture_cd,prefecture,address,address_kana,tel_no,longitude,latitude,floor_area
S12014,千草台店,12,千葉県,千葉県千葉市稲毛区千草台一丁目,チバケンチバシイナゲクチグサダイイッチョウメ,043-123-4003,140.118,35.63559,1698.0
S13002,国分寺店,13,東京都,東京都国分寺市本多二丁目,トウキョウトコクブンジシホンダニチョウメ,042-123-4008,139.4802,35.70566,1735.0
S14010,菊名店,14,神奈川県,神奈川県横浜市港北区菊名一丁目,カナガワケンヨコハマシコウホククキクナイッチョウメ,045-123-4032,139.6326,35.50049,1732.0
S14033,阿久和店,14,神奈川県,神奈川県横浜市瀬谷区阿久和西一丁目,カナガワケンヨコハマシセヤクアクワニシイッチョウメ,045-123-4043,139.4961,35.45918,1495.0
S14036,相模原中央店,14,神奈川県,神奈川県相模原市中央二丁目,カナガワケンサガミハラシチュウオウニチョウメ,042-123-4045,139.3716,35.57327,1679.0


---
> S-017: 顧客データ（customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [32]:
%%sql

SELECT
    *
FROM
    customer
ORDER BY
    birth_day ASC
LIMIT
    10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS003813000014,村山 菜々美,1,女性,1928-11-26,90,182-0007,東京都調布市菊野台**********,S13003,20160214,0-00000000-0
CS026813000004,吉村 朝陽,1,女性,1928-12-14,90,251-0043,神奈川県藤沢市辻堂元町**********,S14026,20150723,0-00000000-0
CS018811000003,熊沢 美里,1,女性,1929-01-07,90,204-0004,東京都清瀬市野塩**********,S13018,20150403,0-00000000-0
CS027803000004,内村 拓郎,0,男性,1929-01-12,90,251-0031,神奈川県藤沢市鵠沼藤が谷**********,S14027,20151227,0-00000000-0
CS013801000003,天野 拓郎,0,男性,1929-01-15,90,274-0824,千葉県船橋市前原東**********,S12013,20160120,0-00000000-0
CS001814000022,鶴田 里穂,1,女性,1929-01-28,90,144-0045,東京都大田区南六郷**********,S13001,20161012,A-20090415-7
CS016815000002,山元 美紀,1,女性,1929-02-22,90,184-0005,東京都小金井市桜町**********,S13016,20150629,C-20090923-C
CS009815000003,中田 里穂,1,女性,1929-04-08,89,154-0014,東京都世田谷区新町**********,S13009,20150421,D-20091021-E
CS012813000013,宇野 南朋,1,女性,1929-04-09,89,231-0806,神奈川県横浜市中区本牧町**********,S14012,20150712,0-00000000-0
CS005813000015,金谷 恵梨香,1,女性,1929-04-09,89,165-0032,東京都中野区鷺宮**********,S13005,20150506,0-00000000-0


---
> S-018: 顧客データ（customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [33]:
%%sql

SELECT
    *
FROM
    customer
ORDER BY
    birth_day DESC
LIMIT
    10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
CS035114000004,大村 美里,1,女性,2007-11-25,11,156-0053,東京都世田谷区桜**********,S13035,20150619,6-20091205-6
CS022103000002,福山 はじめ,9,不明,2007-10-02,11,249-0006,神奈川県逗子市逗子**********,S14022,20160909,0-00000000-0
CS002113000009,柴田 真悠子,1,女性,2007-09-17,11,184-0014,東京都小金井市貫井南町**********,S13002,20160304,0-00000000-0
CS004115000014,松井 京子,1,女性,2007-08-09,11,165-0031,東京都中野区上鷺宮**********,S13004,20161120,1-20081231-1
CS002114000010,山内 遥,1,女性,2007-06-03,11,184-0015,東京都小金井市貫井北町**********,S13002,20160920,6-20100510-1
CS025115000002,小柳 夏希,1,女性,2007-04-18,11,245-0018,神奈川県横浜市泉区上飯田町**********,S14025,20160116,D-20100913-D
CS002113000025,広末 まなみ,1,女性,2007-03-30,12,184-0015,東京都小金井市貫井北町**********,S13002,20171030,0-00000000-0
CS033112000003,長野 美紀,1,女性,2007-03-22,12,245-0051,神奈川県横浜市戸塚区名瀬町**********,S14033,20150606,0-00000000-0
CS007115000006,福岡 瞬,1,女性,2007-03-10,12,285-0845,千葉県佐倉市西志津**********,S12007,20151118,F-20101016-F
CS014113000008,矢口 莉緒,1,女性,2007-03-05,12,260-0041,千葉県千葉市中央区東千葉**********,S12014,20150622,3-20091108-6


---
> S-019: レシート明細データ（receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [36]:
%%sql

SELECT
    customer_id,
    amount,
    rank() OVER(ORDER BY amount DESC)
FROM
    receipt
LIMIT
    10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,amount,rank
CS011415000006,10925,1
ZZ000000000000,6800,2
CS028605000002,5780,3
ZZ000000000000,5480,4
CS015515000034,5480,4
ZZ000000000000,5480,4
ZZ000000000000,5440,7
CS021515000089,5440,7
CS020414000081,5280,9
CS001412000160,5280,9


---
> S-020: レシート明細データ（receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [38]:
%%sql

SELECT
    customer_id,
    amount,
    ROW_NUMBER() OVER(ORDER BY amount DESC)
FROM
    receipt
LIMIT
    10
;

-- 1問目-20問目：30分27秒（2回目のチャレンジ）

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,amount,row_number
CS011415000006,10925,1
ZZ000000000000,6800,2
CS028605000002,5780,3
ZZ000000000000,5480,4
CS015515000034,5480,5
ZZ000000000000,5480,6
ZZ000000000000,5440,7
CS021515000089,5440,8
CS020414000081,5280,9
CS001412000160,5280,10


---
> S-021: レシート明細データ（receipt）に対し、件数をカウントせよ。

In [55]:
%%sql

SELECT
    COUNT(*)
FROM
    receipt
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


count
104681


---
> S-022: レシート明細データ（receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [40]:
%%sql

SELECT
    COUNT(DISTINCT customer_id)
FROM
    receipt
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


count
8307


---
> S-023: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [42]:
%%sql

SELECT
    store_cd,
    SUM(amount) AS sum_amount,
    SUM(quantity) AS sum_quantity
FROM
    receipt
GROUP BY
    store_cd
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,sum_amount,sum_quantity
S12007,638761,2099
S13017,748221,2376
S13043,587895,1881
S13052,100314,250
S13016,793773,2432


---
> S-024: レシート明細データ（receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [43]:
%%sql

SELECT
    customer_id,
    MAX(sales_ymd) AS latest
FROM
    receipt
GROUP BY
    customer_id
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


customer_id,latest
CS001311000059,20180211
CS004614000122,20181228
CS003512000043,20180106
CS011615000061,20190503
CS029212000033,20180621


---
> S-025: レシート明細データ（receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [44]:
%%sql

SELECT
    customer_id,
    MIN(sales_ymd)
FROM
    receipt
GROUP BY
    customer_id
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


customer_id,min
CS001311000059,20180211
CS004614000122,20181228
CS003512000043,20180106
CS011615000061,20190503
CS029212000033,20170318


---
> S-026: レシート明細データ（receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [47]:
%%sql

SELECT
    customer_id,
    MAX(sales_ymd) AS latest,
    MIN(sales_ymd) AS oldest
FROM
    receipt
GROUP BY
    customer_id
HAVING
    MAX(sales_ymd) != MIN(sales_ymd)
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


customer_id,latest,oldest
CS029212000033,20180621,20170318
CS007515000119,20190511,20170201
CS034515000123,20190708,20170527
CS026414000014,20190720,20170718
CS010515000082,20181204,20180518


---
> S-027: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [50]:
%%sql

SELECT
    store_cd,
    AVG(amount) AS avg_amount
FROM
    receipt
GROUP BY
    store_cd
ORDER BY
    avg_amount DESC
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,avg_amount
S13052,402.8674698795180723
S13015,351.1119604316546763
S13003,350.9155188246097337
S14010,348.7912621359223301
S13001,348.4703862660944206


---
> S-028: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [52]:
%%sql

SELECT
    store_cd,
    PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY amount) AS amount_50per
FROM
    receipt
GROUP BY
    store_cd
ORDER BY 
    amount_50per DESC
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,amount_50per
S13052,190.0
S14010,188.0
S14050,185.0
S13003,180.0
S13018,180.0


---
> S-029: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [54]:
%%sql

SELECT
    store_cd,
    MODE() WITHIN GROUP(ORDER BY product_cd)
FROM
    receipt
GROUP BY
    store_cd
LIMIT
    10
;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


store_cd,mode
S12007,P060303001
S12013,P060303001
S12014,P060303001
S12029,P060303001
S12030,P060303001
S13001,P060303001
S13002,P060303001
S13003,P071401001
S13004,P060303001
S13005,P040503001


---
> S-030: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [58]:
%%sql

SELECT
    store_cd,
    VAR_POP(amount) val_amount
FROM
    receipt
GROUP BY
    store_cd
ORDER BY 
    val_amount DESC
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,val_amount
S13052,440088.701311269173
S14011,306314.558163888889
S14034,296920.081011283873
S13001,295431.993329035348
S13015,295294.361115940880


---
> S-031: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

In [60]:
%%sql

SELECT
    store_cd,
    STDDEV_POP(amount) AS std_amount
FROM
    receipt
GROUP BY
    store_cd
ORDER BY
    std_amount DESC
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,std_amount
S13052,663.391815830787
S14011,553.456916267101
S14034,544.903735545357
S13001,543.536561170484
S13015,543.409938366921


---
> S-032: レシート明細データ（receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [63]:
%%sql

SELECT
    PERCENTILE_CONT(0.25) WITHIN GROUP(ORDER BY amount) AS amount_25per,
    PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY amount) AS amount_50per,
    PERCENTILE_CONT(0.75) WITHIN GROUP(ORDER BY amount) AS amount_75per,
    PERCENTILE_CONT(1.0) WITHIN GROUP(ORDER BY amount) AS amount_100per
FROM 
    receipt
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


amount_25per,amount_50per,amount_75per,amount_100per
102.0,170.0,288.0,10925.0


---
> S-033: レシート明細データ（receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [64]:
%%sql

SELECT
    store_cd,
    AVG(amount) AS avg_amount
FROM
    receipt
GROUP BY
    store_cd
HAVING
    AVG(amount) >= 330
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


store_cd,avg_amount
S13052,402.8674698795180723
S13019,330.2086158755484643
S13003,350.9155188246097337
S14045,330.0820734341252700
S13004,330.9439490445859873


---
> S-034: レシート明細データ（receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [67]:
%%sql

WITH sum_amount_by_customer AS (
    SELECT
        customer_id,
        SUM(amount) AS sum_amount
    FROM
        receipt
    WHERE
        customer_id NOT LIKE 'Z%'
    GROUP BY
        customer_id
)
SELECT
    AVG(sum_amount)
FROM
    sum_amount_by_customer
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


avg
2547.7422345292559595


---
> S-035: レシート明細データ（receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [73]:
%%sql

--STEP1, customer_idについて、Zから始まる顧客を除く
--STEP2, amountをcustomer_idごとに合計
--STEP3, amountの全体平均を求める
--STEP4, 全体平均amountとcustomerごとのamount合計値を比較して、平均以上に購入しているもののみ表示する

WITH customer_data AS (
    SELECT
        customer_id,
        amount
    FROM
        receipt
    WHERE
        customer_id NOT LIKE 'Z%'
), sum_amount_by_customer AS (
    SELECT
        customer_id,
        sum(amount) sum_amount
    FROM
        customer_data
    GROUP BY
        customer_id
), avg_amount_for_all AS (
    SELECT
        avg(sum_amount) AS avg_amount
    FROM
        sum_amount_by_customer
)
SELECT
    s.customer_id,
    s.sum_amount
FROM
    sum_amount_by_customer s,
    avg_amount_for_all a
WHERE
    s.sum_amount >= a.avg_amount
LIMIT
    5;    

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


customer_id,sum_amount
CS029212000033,3604
CS007515000119,7157
CS034515000123,3699
CS026414000014,6671
CS007615000070,2975


---
> S-036: レシート明細データ（receipt）と店舗データ（store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [78]:
%%sql

SELECT
    r.*,
    s.store_name
FROM
    receipt r
INNER JOIN
    store s
ON
    r.store_cd = s.store_cd
LIMIT
    5
;  

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount,store_name
20181103,1541203200,S14006,112,1,CS006214000001,P070305012,1,158,葛が谷店
20181118,1542499200,S13008,1132,2,CS008415000097,P070701017,1,81,成城店
20170712,1499817600,S14028,1102,1,CS028414000014,P060101005,1,170,二ツ橋店
20190205,1549324800,S14042,1132,1,ZZ000000000000,P050301001,1,25,新山下店
20180821,1534809600,S14025,1102,2,CS025415000050,P060102007,1,90,大和店


---
> S-037: 商品データ（product）とカテゴリデータ（category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [81]:
%%sql

SELECT
    p.*,
    c.category_small_name
FROM
    product p
INNER JOIN
    category c
ON
    p.category_small_cd = c.category_small_cd
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost,category_small_name
P040101001,04,0401,040101,198,149,弁当類
P040101002,04,0401,040101,218,164,弁当類
P040101003,04,0401,040101,230,173,弁当類
P040101004,04,0401,040101,248,186,弁当類
P040101005,04,0401,040101,268,201,弁当類


---
> S-038: 顧客データ（customer）とレシート明細データ（receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [91]:
%%sql

-- STEP1, 顧客ごとの売上金額合計を出す
-- STEP2, 対象を女性、非会員は除外する
-- STEP3, 顧客データとレシートデータ結合

WITH customer_amount AS (
    SELECT
        customer_id,
        SUM(amount) sum_amount
    FROM
        receipt
    GROUP BY
        customer_id
), customer_data AS (
    SELECT
        customer_id,
        gender_cd
    FROM
        customer
    WHERE
        customer_id NOT LIKE 'Z%'
        AND gender_cd = '1'
)
SELECT
    cd.customer_id,
    COALESCE(ca.sum_amount, 0)
FROM
    customer_data cd
LEFT JOIN
    customer_amount ca
ON
    cd.customer_id = ca.customer_id
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


customer_id,coalesce
CS021313000114,0
CS031415000172,5088
CS028811000001,0
CS001215000145,875
CS015414000103,3122


---
> S-039: レシート明細データ（receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [101]:
%%sql

WITH customer_data AS (
    SELECT
        *
    FROM
        receipt
    WHERE
        customer_id NOT LIKE 'Z%'
), customer_days AS (
    SELECT 
        customer_id,
        COUNT(DISTINCT sales_ymd) AS days
    FROM
        customer_data
    GROUP BY
        customer_id
    ORDER BY
        days DESC
    LIMIT
        20
), customer_amount AS (
    SELECT
        customer_id,
        SUM(amount) sum_amount
    FROM
        customer_data
    GROUP BY
        customer_id
    ORDER BY
        sum_amount DESC
    LIMIT
        20
)
SELECT
    COALESCE(cd.customer_id, ca.customer_id) AS customer_id,
    cd.days,
    ca.sum_amount
FROM
    customer_days cd
FULL OUTER JOIN
    customer_amount ca
ON 
    cd.customer_id = ca.customer_id
;

*  postgresql://padawan:***@db:5432/dsdojo_db
34 rows affected.


customer_id,days,sum_amount
CS040214000008,23,None
CS015415000185,22,20153
CS010214000010,22,18585
CS028415000007,21,19127
CS010214000002,21,None
CS017415000097,20,23086
CS016415000141,20,18372
CS021514000045,19,None
CS022515000226,19,None
CS031414000051,19,19202


---
> S-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（store）と商品データ（product）を直積し、件数を計算せよ。

In [103]:
%%sql

SELECT
    COUNT(*)
FROM
    store
CROSS JOIN
    product
LIMIT
    5
;

-- 21問目-40問目：1時間38分20秒（2回目のチャレンジ）

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


count
531590


---
> S-041: レシート明細データ（receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [7]:
%%sql

WITH sales_amount_by_date AS (
    SELECT
        sales_ymd,
        SUM(amount) AS sum_amount
    FROM
        receipt
    GROUP BY
        sales_ymd
)
SELECT 
    sales_ymd,
    sum_amount,
    LAG(sales_ymd, 1) OVER(ORDER BY sales_ymd ASC) AS sales_ymd_lag1,
    LAG(sum_amount, 1) OVER(ORDER BY sales_ymd ASC) AS sum_amount_lag1,
    sum_amount - LAG(sum_amount, 1) OVER(ORDER BY sales_ymd ASC) AS diff
FROM
    sales_amount_by_date
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


sales_ymd,sum_amount,sales_ymd_lag1,sum_amount_lag1,diff
20170101,33723,None,None,None
20170102,24165,20170101,33723,-9558
20170103,27503,20170102,24165,3338


---
> S-042: レシート明細データ（receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [9]:
%%sql

--amountを日付ごとに集計
--前回、前々回、3回前月のテーブルをSELECT

WITH sales_amount_by_date AS (
    SELECT
        sales_ymd,
        SUM(amount) AS sum_amount
    FROM
        receipt
    GROUP BY
        sales_ymd
)
SELECT
    sales_ymd,
    sum_amount,
    LAG(sum_amount, 1) OVER(ORDER BY sales_ymd ASC) AS sum_amount_lag1,
    LAG(sum_amount, 2) OVER(ORDER BY sales_ymd ASC) AS sum_amount_lag2,
    LAG(sum_amount, 3) OVER(ORDER BY sales_ymd ASC) AS sum_amount_lag3
FROM
    sales_amount_by_date
LIMIT
    3;
    

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


sales_ymd,sum_amount,sum_amount_lag1,sum_amount_lag2,sum_amount_lag3
20170101,33723,None,None,None
20170102,24165,33723,None,None
20170103,27503,24165,33723,None


---
> S-043: レシート明細データ（receipt）と顧客データ（customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [30]:
%%sql

--customer_idごとにamountを合計(receipt tableから)
--customer tableとcustomer_idで結合(gender, ageを取得)
--ageを10年代ずつに変換
--CASE文でgenderの番号ごとに1ならsum_amountをAS maleのコラムに加算。SELECTでeraも書いておく。

DROP TABLE IF EXISTS customer_1;

CREATE TABLE customer_1 AS
    WITH sum_amount_by_customer AS (
        SELECT
            c.gender_cd,
            TRUNC(c.age / 10) * 10 AS era,
            SUM(r.amount) AS sum_amount
        FROM
            receipt r
        LEFT JOIN
            customer c
        ON
            r.customer_id = c.customer_id
        GROUP BY
            gender_cd,
            era
    )
    SELECT
        era,
        SUM(CASE WHEN gender_cd = '0' THEN sum_amount END) AS male,
        SUM(CASE WHEN gender_cd = '1' THEN sum_amount END) AS female,
        SUM(CASE WHEN gender_cd = '9' THEN sum_amount END) AS unknown
    FROM
        sum_amount_by_customer
    GROUP BY
        era
    ORDER BY
        era
;

SELECT
    *
FROM
    customer_1
LIMIT
    3
;

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
10 rows affected.
3 rows affected.


era,male,female,unknown
10.0,1591,149836,4317
20.0,72940,1363724,44328
30.0,177322,693047,50441


---
> S-044: 043で作成した売上サマリデータ（sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [36]:
%%sql

SELECT 
    era, 
    '00' AS gender_cd, 
    male AS amount
FROM 
    customer_1
UNION
SELECT
    era,
    '01' AS gender_cd,
    female AS amount
FROM
    customer_1
UNION
SELECT
    era,
    '99' AS gender_cd,
    unknown AS amount
FROM
    customer_1
;
    


*  postgresql://padawan:***@db:5432/dsdojo_db
30 rows affected.


era,gender_cd,amount
20.0,99,44328
90.0,00,None
None,00,None
60.0,01,987741
50.0,00,54320
80.0,99,5111
20.0,00,72940
10.0,99,4317
30.0,01,693047
None,99,None


---
> S-045: 顧客データ（customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [38]:
%%sql

SELECT
    customer_id,
    birth_day,
    TO_CHAR(birth_day, 'YYYYMMDD') AS birth_day_varchar
FROM
    customer
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,birth_day,birth_day_varchar
CS021313000114,1981-04-29,19810429
CS037613000071,1952-04-01,19520401
CS031415000172,1976-10-04,19761004


---
> S-046: 顧客データ（customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [39]:
%%sql

SELECT
    customer_id,
    application_date,
    TO_DATE(application_date, 'YYYYMMDD') AS application_date_timestamp
FROM
    customer
LIMIT
    3
;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,application_date,application_date_timestamp
CS021313000114,20150905,2015-09-05
CS037613000071,20150414,2015-04-14
CS031415000172,20150529,2015-05-29


---
> S-047: レシート明細データ（receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [43]:
%%sql

SELECT
    receipt_no,
    receipt_sub_no,
    sales_ymd,
    TO_DATE(CAST(sales_ymd AS VARCHAR), 'YYYYMMDD') AS sales_ymd_datetype
FROM
    receipt
LIMIT
    3
;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


receipt_no,receipt_sub_no,sales_ymd,sales_ymd_datetype
112,1,20181103,2018-11-03
1132,2,20181118,2018-11-18
1102,1,20170712,2017-07-12


---
> S-048: レシート明細データ（receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [47]:
%%sql

SELECT
    receipt_no,
    receipt_sub_no,
    sales_epoch,
    CAST(TO_TIMESTAMP(sales_epoch) AS DATE) AS sales_date
FROM
    receipt
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


receipt_no,receipt_sub_no,sales_epoch,sales_date
112,1,1541203200,2018-11-03
1132,2,1542499200,2018-11-18
1102,1,1499817600,2017-07-12


---
> S-049: レシート明細データ（receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [54]:
%%sql

SELECT
    receipt_no,
    receipt_sub_no,
    sales_epoch,
    EXTRACT(YEAR FROM 
    TO_TIMESTAMP(sales_epoch)
    ) AS sales_year
FROM
    receipt
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


receipt_no,receipt_sub_no,sales_epoch,sales_year
112,1,1541203200,2018
1132,2,1542499200,2018
1102,1,1499817600,2017


---
> S-050: レシート明細データ（receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [62]:
%%sql

SELECT
    receipt_no,
    receipt_sub_no,
    sales_epoch,
    TO_CHAR(
        EXTRACT(MONTH FROM TO_TIMESTAMP(sales_epoch)),
        'FM00') AS sales_month
FROM
    receipt
LIMIT
    3;


*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


receipt_no,receipt_sub_no,sales_epoch,sales_month
112,1,1541203200,11
1132,2,1542499200,11
1102,1,1499817600,07


---
> S-051: レシート明細データ（receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [65]:
%%sql


SELECT
    receipt_no,
    receipt_sub_no,
    sales_epoch,
    TO_CHAR(
        EXTRACT(DAY FROM TO_TIMESTAMP(sales_epoch)), 'FM00'
    ) AS sales_date
FROM
    receiptS
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


receipt_no,receipt_sub_no,sales_epoch,sales_date
112,1,1541203200,03
1132,2,1542499200,18
1102,1,1499817600,12


---
> S-052: レシート明細データ（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [67]:
%%sql

--suctomer_idごとに売上金額を合計
--フラッグ作成

WITH sales_amount AS (
    SELECT
        customer_id,
        SUM(amount) AS sum_amount
    FROM
        receipt
    WHERE
        customer_id NOT LIKE 'Z%'
    GROUP BY
        customer_id
)
SELECT
    customer_id,
    sum_amount,
    CASE
        WHEN sum_amount <= 2000 THEN 0
        WHEN sum_amount > 2000 THEN 1
        ELSE 0
        END AS sales_flg
FROM
    sales_amount
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,sum_amount,sales_flg
CS001311000059,2302,1
CS004614000122,248,0
CS003512000043,298,0


---
> S-053: 顧客データ（customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [73]:
%%sql

SELECT
    customer_id, COUNT(*) AS cnt
FROM
    customer
GROUP BY
    customer_id
ORDER BY
    cnt DESC
LIMIT
    3;
--customer_idは重複無し。

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,cnt
CS019612000010,1
CS004215000095,1
CS012502000051,1


In [79]:
%%sql

SELECT
    customer_id,
    postal_cd,
    CASE
        WHEN CAST(SUBSTR(postal_cd, 1, 3) AS INTEGER) BETWEEN 100 AND 209 THEN 1
        ELSE 0 
        END AS postal_cd_flg
FROM
    customer
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,postal_cd,postal_cd_flg
CS021313000114,259-1113,0
CS037613000071,136-0076,1
CS031415000172,151-0053,1


---
> S-054: 顧客データ（customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [81]:
%%sql

SELECT
    customer_id,
    address,
    CASE SUBSTR(address, 1, 3)
        WHEN '埼玉県' THEN 11
        WHEN '千葉県' THEN 12
        WHEN '東京都' THEN 13
        WHEN '神奈川' THEN 14
    END AS address_cd
FROM
    customer
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,address,address_cd
CS021313000114,神奈川県伊勢原市粟窪**********,14
CS037613000071,東京都江東区南砂**********,13
CS031415000172,東京都渋谷区代々木**********,13


---
> S-055: レシート明細（receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [91]:
%%sql

--customer_idごとにamountを集計
--CASEを使ってcategory分類

WITH sales_amount AS (
    SELECT
        customer_id,
        SUM(amount) AS sum_amount
    FROM
        receipt
    GROUP BY
        customer_id
), stats AS (
    SELECT
        PERCENTILE_CONT(0.25) WITHIN GROUP(ORDER BY sum_amount) AS amount_25pct,
        PERCENTILE_CONT(0.50) WITHIN GROUP(ORDER BY sum_amount) AS amount_50pct,
        PERCENTILE_CONT(0.75) WITHIN GROUP(ORDER BY sum_amount) AS amount_75pct,
        PERCENTILE_CONT(.0) WITHIN GROUP(ORDER BY sum_amount) AS amount_100pct
    FROM
        sales_amount
)
SELECT
    sa.customer_id,
    sa.sum_amount,
    st.amount_25pct,
    st.amount_50pct,
    st.amount_75pct,
    st.amount_100pct,
    CASE
        WHEN sa.sum_amount < st.amount_25pct THEN 1 
        WHEN st.amount_25pct <= sa.sum_amount AND sa.sum_amount < st.amount_50pct THEN 2 
        WHEN st.amount_50pct <= sa.sum_amount AND sa.sum_amount < st.amount_75pct THEN 3 
        ELSE 4
        END AS sum_amount_category
FROM
    sales_amount sa
CROSS JOIN
    stats st
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,sum_amount,amount_25pct,amount_50pct,amount_75pct,amount_100pct,sum_amount_category
CS001311000059,2302,548.5,1478.0,3651.0,70.0,3
CS004614000122,248,548.5,1478.0,3651.0,70.0,1
CS003512000043,298,548.5,1478.0,3651.0,70.0,1


---
> S-056: 顧客データ（customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [95]:
%%sql

SELECT
    customer_id,
    birth_day,
    age,
    LEAST(TRUNC(age/10) * 10, 60) AS era
FROM
    customer
LIMIT
    10;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,birth_day,age,era
CS021313000114,1981-04-29,37,30.0
CS037613000071,1952-04-01,66,60.0
CS031415000172,1976-10-04,42,40.0
CS028811000001,1933-03-27,86,60.0
CS001215000145,1995-03-29,24,20.0
CS020401000016,1974-09-15,44,40.0
CS015414000103,1977-08-09,41,40.0
CS029403000008,1973-08-17,45,40.0
CS015804000004,1931-05-02,87,60.0
CS033513000180,1962-07-11,56,50.0


---
> S-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [101]:
%%sql

WITH era_tbl AS (
    SELECT
        customer_id,
        birth_day,
        age,
        LEAST(TRUNC(age/10) * 10, 60) AS era,
        gender_cd
    FROM
        customer
)
SELECT
        customer_id,
        birth_day,
        age,
        era,
        gender_cd,
        CONCAT(CAST(era AS VARCHAR),CAST(gender_cd AS VARCHAR)) AS era_gender_cd
FROM
    era_tbl
LIMIT
    3;


*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,birth_day,age,era,gender_cd,era_gender_cd
CS021313000114,1981-04-29,37,30.0,1,301
CS037613000071,1952-04-01,66,60.0,9,609
CS031415000172,1976-10-04,42,40.0,1,401


---
> S-058: 顧客データ（customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [109]:
%%sql

SELECT
    customer_id,
    gender_cd,
    CASE WHEN gender_cd = '0' THEN 1 ELSE 0 END AS male,
    CASE WHEN gender_cd = '1' THEN 1 ELSE 0 END AS female,
    CASE WHEN gender_cd = '9' THEN 1 ELSE 0 END AS unknown
FROM
    customer
LIMIT
    3;


*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,gender_cd,male,female,unknown
CS021313000114,1,0,1,0
CS037613000071,9,0,0,1
CS031415000172,1,0,1,0


---
> S-059: レシート明細データ（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [114]:
%%sql

--customer_idごとにamountを集計, 非会員は除外
--statsで統計値集計
--各customer_idのamountを標準化

WITH sales_amount AS (
    SELECT
        customer_id,
        SUM(amount) AS sum_amount
    FROM
        receipt
    WHERE
        customer_id NOT LIKE 'Z%'
    GROUP BY
        customer_id
), stats_tbl AS (
    SELECT
        AVG(sum_amount) AS avg_amount,
        STDDEV_POP(sum_amount) AS stddev_amount
    FROM
        sales_amount    
)
SELECT
    sa.customer_id,
    sa.sum_amount,
    st.avg_amount,
    st.stddev_amount,
    (sa.sum_amount - st.avg_amount) / st.stddev_amount AS std_amount
FROM
    sales_amount sa
CROSS JOIN
    stats_tbl st
LIMIT
    10;


*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,avg_amount,stddev_amount,std_amount
CS001311000059,2302,2547.7422345292559595,2720.510255798560,-0.09032946448390523062
CS004614000122,248,2547.7422345292559595,2720.510255798560,-0.84533488878695858131
CS003512000043,298,2547.7422345292559595,2720.510255798560,-0.82695598361891930792
CS011615000061,246,2547.7422345292559595,2720.510255798560,-0.84607004499368015224
CS029212000033,3604,2547.7422345292559595,2720.510255798560,0.38825722609183744835
CS007515000119,7157,2547.7422345292559595,2720.510255798560,1.6942622273327082
CS034515000123,3699,2547.7422345292559595,2720.510255798560,0.42317714591111206778
CS004315000058,490,2547.7422345292559595,2720.510255798560,-0.75638098777364849812
CS026414000014,6671,2547.7422345292559595,2720.510255798560,1.5156192690993665
CS001615000099,768,2547.7422345292559595,2720.510255798560,-0.65419427503935013810


---
> S-060: レシート明細データ（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [118]:
%%sql

--customer_idごとにamountを集計,非会員を除外
--stats tblで統計値計算
--正規化適用

WITH sales_amount AS (
    SELECT
        customer_id,
        SUM(amount) AS sum_amount
    FROM
        receipt
    WHERE
        customer_id NOT LIKE 'Z%'
    GROUP BY
        customer_id
), stats_tbl AS (
    SELECT
        MIN(sum_amount) AS min_amount,
        MAX(sum_amount) AS max_amount
    FROM
        sales_amount
)
SELECT
    customer_id,
    sum_amount,
    min_amount,
    max_amount,
    (sum_amount - min_amount) * 1.0 / (max_amount - min_amount) AS std_amount
FROM
    sales_amount
CROSS JOIN
    stats_tbl
LIMIT
    3;

-- 41問目-60問目：2時間04分07秒（2回目のチャレンジ）

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,sum_amount,min_amount,max_amount,std_amount
CS001311000059,2302,70,23086,0.09697601668404588113
CS004614000122,248,70,23086,0.00773375043448036149
CS003512000043,298,70,23086,0.00990615224191866528


---
> S-061: レシート明細データ（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [127]:
%%sql

--customer_idごとにamountを集計,非会員を除外
--常用対数化

WITH sales_amount AS (
    SELECT
        customer_id,
        SUM(amount) AS sum_amount
    FROM
        receipt
    WHERE
        customer_id NOT LIKE 'Z%'
    GROUP BY
        customer_id
)
SELECT
    customer_id,
    sum_amount,
    LOG(sum_amount + 0.5) AS log_amount
FROM
    sales_amount
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,sum_amount,log_amount
CS001311000059,2302,3.3621996388688865
CS004614000122,248,2.3953263930693509
CS003512000043,298,2.4749443354653879


---
> S-062: レシート明細データ（receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [128]:
%%sql


--customer_idごとにamountを集計,非会員を除外
--常用対数化

WITH sales_amount AS (
    SELECT
        customer_id,
        SUM(amount) AS sum_amount
    FROM
        receipt
    WHERE
        customer_id NOT LIKE 'Z%'
    GROUP BY
        customer_id
)
SELECT
    customer_id,
    sum_amount,
    LN(sum_amount + 0.5) AS nlog_amount
FROM
    sales_amount
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,sum_amount,nlog_amount
CS001311000059,2302,7.7417507681294619
CS004614000122,248,5.5154428455366834
CS003512000043,298,5.6987699328326568


---
> S-063: 商品データ（product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [124]:
%%sql

SELECT
    product_cd,
    unit_price,
    unit_cost,
    unit_price - unit_cost AS unit_profit
FROM
    product
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


product_cd,unit_price,unit_cost,unit_profit
P040101001,198,149,49
P040101002,218,164,54
P040101003,230,173,57


---
> S-064: 商品データ（product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [129]:
%%sql
--各productの利益率を計算
--利益率の全体平均を計算

WITH product_profit AS (
    SELECT
        product_cd,
        unit_price,
        unit_cost,
        (unit_price - unit_cost) * 1.0 / unit_price AS unit_profit_rate
    FROM
        product
)
SELECT
    AVG(unit_profit_rate)
FROM
    product_profit
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


avg
0.24911389885177001279


---
> S-065: 商品データ（product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [132]:
%%sql

SELECT
    product_cd,
    unit_price,
    unit_cost,
    FLOOR(unit_cost / 0.7) AS new_price
FROM
    product
LIMIT
    3;


*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


product_cd,unit_price,unit_cost,new_price
P040101001,198,149,212
P040101002,218,164,234
P040101003,230,173,247


---
> S-066: 商品データ（product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [134]:
%%sql

SELECT
    product_cd,
    unit_price,
    unit_cost,
    ROUND(unit_cost / 0.7) AS new_price,
    (ROUND(unit_cost / 0.7) - unit_cost) * 1.0/ ROUND(unit_cost / 0.7) AS new_profit_rate
FROM
    product
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


product_cd,unit_price,unit_cost,new_price,new_profit_rate
P040101001,198,149,213,0.30046948356807511737
P040101002,218,164,234,0.29914529914529914530
P040101003,230,173,247,0.29959514170040485830


---
> S-067: 商品データ（product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [135]:
%%sql

SELECT
    product_cd,
    unit_price,
    unit_cost,
    ROUND(unit_cost / 0.7) AS new_price,
    (CEIL(unit_cost / 0.7) - unit_cost) * 1.0/ ROUND(unit_cost / 0.7) AS new_profit_rate
FROM
    product
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


product_cd,unit_price,unit_cost,new_price,new_profit_rate
P040101001,198,149,213,0.30046948356807511737
P040101002,218,164,234,0.30341880341880341880
P040101003,230,173,247,0.30364372469635627530


---
> S-068: 商品データ（product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [138]:
%%sql

SELECT
    product_cd,
    unit_price,
    FLOOR(unit_price * 1.1) AS unit_price_wit_tax
FROM
    product
LIMIT
    3;


*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


product_cd,unit_price,unit_price_wit_tax
P040101001,198,217
P040101002,218,239
P040101003,230,253


---
> S-069: レシート明細データ（receipt）と商品データ（product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [143]:
%%sql

--全商品の売上金額合計をcustomer_idごとに計算
--商品07の売上金額合計をcustomer_idごとに計算、07の売上がある顧客だけ計算
--夫々の比率を求める

WITH sales_amount_all AS (
    SELECT
        customer_id,
        SUM(amount) AS sum_amount_all
    FROM
        receipt
    GROUP BY
        customer_id
), sales_amount_07 AS (
    SELECT
        customer_id,
        SUM(amount) AS sum_amount_07
    FROM
        receipt r
    INNER JOIN
        product p
    ON
        r.product_cd = p.product_cd
    WHERE
        p.category_major_cd = '07'
    GROUP BY
        customer_id
)
SELECT
    sall.customer_id,
    sall.sum_amount_all,
    s07.sum_amount_07,
    s07.sum_amount_07 * 1.0 / sall.sum_amount_all AS ratio
FROM
    sales_amount_all sall
INNER JOIN
    sales_amount_07 s07
ON
    sall.customer_id = s07.customer_id
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,sum_amount_all,sum_amount_07,ratio
CS001311000059,2302,102,0.04430929626411815812
CS011615000061,246,98,0.39837398373983739837
CS029212000033,3604,3604,1.00000000000000000000


---
> S-070: レシート明細データ（receipt）の売上日（sales_ymd）に対し、顧客データ（customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [151]:
%%sql

SELECT
    r.customer_id,
    r.sales_ymd,
    c.application_date,
    EXTRACT(DAY FROM
    (TO_TIMESTAMP(CAST(r.sales_ymd AS VARCHAR), 'YYYYMMDD')
    - TO_TIMESTAMP(c.application_date, 'YYYYMMDD'))
           ) AS elapsed_days
FROM
    receipt r
INNER JOIN
    customer c
ON
    r.customer_id = c.customer_id
LIMIT
    3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,sales_ymd,application_date,elapsed_days
CS006214000001,20181103,20150201,1371
CS008415000097,20181118,20150322,1337
CS028414000014,20170712,20150711,732


---
> S-071: レシート明細データ（receipt）の売上日（sales_ymd）に対し、顧客データ（customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [161]:
%%sql

WITH elapsed_age_tbl AS (
    SELECT
        r.customer_id,
        r.sales_ymd,
        c.application_date,
        AGE(
        TO_TIMESTAMP(CAST(r.sales_ymd AS VARCHAR), 'YYYYMMDD'),
        TO_TIMESTAMP(c.application_date, 'YYYYMMDD')
               ) AS elapsed
    FROM
        receipt r
    INNER JOIN
        customer c
    ON
        r.customer_id = c.customer_id
)
SELECT
    customer_id,
    sales_ymd,
    application_date,
    elapsed,
    EXTRACT(YEAR FROM elapsed) * 12
    + EXTRACT(MONTH FROM elapsed) AS elapsed_month
FROM
    elapsed_age_tbl
LIMIT
    3;


*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,sales_ymd,application_date,elapsed,elapsed_month
CS006214000001,20181103,20150201,"1367 days, 0:00:00",45
CS008415000097,20181118,20150322,"1332 days, 0:00:00",43
CS028414000014,20170712,20150711,"731 days, 0:00:00",24


---
> S-072: レシート明細データ（receipt）の売上日（sales_ymd）に対し、顧客データ（customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [162]:
%%sql

WITH elapsed_age_tbl AS (
    SELECT
        r.customer_id,
        r.sales_ymd,
        c.application_date,
        AGE(
        TO_TIMESTAMP(CAST(r.sales_ymd AS VARCHAR), 'YYYYMMDD'),
        TO_TIMESTAMP(c.application_date, 'YYYYMMDD')
               ) AS elapsed
    FROM
        receipt r
    INNER JOIN
        customer c
    ON
        r.customer_id = c.customer_id
)
SELECT
    customer_id,
    sales_ymd,
    application_date,
    elapsed,
    EXTRACT(YEAR FROM elapsed) AS elapsed_year
FROM
    elapsed_age_tbl
LIMIT
    3;


*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,sales_ymd,application_date,elapsed,elapsed_year
CS006214000001,20181103,20150201,"1367 days, 0:00:00",3
CS008415000097,20181118,20150322,"1332 days, 0:00:00",3
CS028414000014,20170712,20150711,"731 days, 0:00:00",2


---
> S-073: レシート明細データ（receipt）の売上日（sales_ymd）に対し、顧客データ（customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [174]:
%%sql

WITH distinct_receipt AS (
    SELECT DISTINCT
        sales_ymd,
        customer_id
    FROM
        receipt
)
SELECT
    d.customer_id,
    d.sales_ymd,
    c.application_date,
    EXTRACT(epoch FROM
    AGE(
    TO_TIMESTAMP(CAST(d.sales_ymd AS VARCHAR), 'YYYYMMDD'),
    TO_TIMESTAMP(c.application_date, 'YYYYMMDD')
    )) AS elapsed_epoch

FROM
    distinct_receipt d
INNER JOIN
    customer c
ON
    d.customer_id = c.customer_id
LIMIT
    3;


*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,sales_ymd,application_date,elapsed_epoch
CS020414000081,20190429,20150301,131241600.000000
CS011415000188,20171220,20150506,82468800.000000
CS014514000054,20180525,20150803,88344000.000000


---
> S-074: レシート明細データ（receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [183]:
%%sql

WITH elapsed_tbl AS (
    SELECT
        TO_DATE(CAST(sales_ymd AS VARCHAR), 'YYYYMMDD') AS sales_date,
        EXTRACT(DOW FROM (
        TO_DATE(CAST(sales_ymd AS VARCHAR), 'YYYYMMDD') - 1 
        )) AS elapsed_days
    FROM
        receipt
)
SELECT
    sales_date,
    elapsed_days,
    sales_date - CAST(elapsed_days AS INTEGER) AS last_monday
FROM
    elapsed_tbl
LIMIT
    3;


*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


sales_date,elapsed_days,last_monday
2018-11-03,5,2018-10-29
2018-11-18,6,2018-11-12
2017-07-12,2,2017-07-10


---
> S-075: 顧客データ（customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [190]:
%%sql

SELECT SETSEED(0.1);
WITH row_tbl AS (
    SELECT
        ROW_NUMBER() OVER(ORDER BY RANDOM()) AS row,
        *
    FROM
        receipt
)
SELECT
    *
FROM
    row_tbl
WHERE
    row <= (
        SELECT
            COUNT(*)
        FROM
            receipt
    ) * 0.01
LIMIT
    3
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.
3 rows affected.


row,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
1,20190322,1553212800,S13038,1182,2,CS038515000215,P070310003,1,228
2,20190330,1553904000,S13016,1102,1,ZZ000000000000,P050701006,1,108
3,20170828,1503878400,S13005,1102,2,ZZ000000000000,P070506003,1,188


---
> S-076: 顧客データ（customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [202]:
%%sql

SELECT SETSEED(0.1);

WITH row_tbl AS (
    SELECT
        ROW_NUMBER() OVER(PARTITION BY gender_cd ORDER BY RANDOM()) AS row,
        COUNT(*) OVER(PARTITION BY gender_cd) AS cnt,
        *
    FROM
        customer
)
SELECT
    gender_cd,
    COUNT(*)
FROM
    row_tbl
WHERE
    row <= cnt * 0.1
GROUP BY
    gender_cd
;



*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.
3 rows affected.


gender_cd,count
9,107
0,298
1,1791


---
> S-077: レシート明細データ（receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [213]:
%%sql

WITH sales_amount AS (
    SELECT
        customer_id,
        SUM(amount) AS sum_amount,
        LOG(SUM(amount) + 0.5) AS log_amount
    FROM
        receipt
    GROUP BY
        customer_id
), stats_tbl AS (
    SELECT
        AVG(log_amount) AS avg_amount,
        stddev_POP(log_amount) AS std_amount
    FROM
        sales_amount
)
SELECT
    sa.customer_id,
    sa.sum_amount,
    sa.log_amount,
    st.avg_amount
FROM
    sales_amount sa
CROSS JOIN
    stats_tbl st
WHERE
    ABS(sa.log_amount - st.avg_amount) > 3 * std_amount
LIMIT
    3
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


customer_id,sum_amount,log_amount,avg_amount
ZZ000000000000,12395003,7.093246653736391,3.1577235163277617


---
> S-078: レシート明細データ（receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [218]:
%%sql

--集計、非会員を除外
--統計値集計
--外れ値検出

WITH sales_amount AS (
    SELECT
        customer_id,
        SUM(amount) AS sum_amount
    FROM
        receipt
    WHERE
        customer_id NOT LIKE 'Z%'
    GROUP BY
        customer_id
), stats_tbl AS (
    SELECT
        PERCENTILE_CONT(0.25) WITHIN GROUP(ORDER BY sum_amount) AS sum_amount_25pct,
        PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY sum_amount) AS sum_amount_50pct,
        PERCENTILE_CONT(0.75) WITHIN GROUP(ORDER BY sum_amount) AS sum_amount_75pct,
        PERCENTILE_CONT(1.0) WITHIN GROUP(ORDER BY sum_amount) AS sum_amount_100pct
    FROM
        sales_amount
)
SELECT
    sa.customer_id,
    sa.sum_amount,
    st.sum_amount_25pct,
    st.sum_amount_50pct,
    st.sum_amount_75pct,
    st.sum_amount_100pct
FROM
    sales_amount sa
CROSS JOIN
    stats_tbl st
WHERE
    sa.sum_amount < st.sum_amount_25pct - 1.5 * (st.sum_amount_75pct - st.sum_amount_25pct)
    OR st.sum_amount_75pct + 1.5 * (st.sum_amount_75pct - st.sum_amount_25pct) < sa.sum_amount

LIMIT
    10;

*  postgresql://padawan:***@db:5432/dsdojo_db
10 rows affected.


customer_id,sum_amount,sum_amount_25pct,sum_amount_50pct,sum_amount_75pct,sum_amount_100pct
CS013415000226,8362,548.25,1478.0,3649.75,23086.0
CS011415000041,9454,548.25,1478.0,3649.75,23086.0
CS014514000004,8872,548.25,1478.0,3649.75,23086.0
CS021514000008,12839,548.25,1478.0,3649.75,23086.0
CS014515000007,9763,548.25,1478.0,3649.75,23086.0
CS040415000220,10158,548.25,1478.0,3649.75,23086.0
CS028415000161,8465,548.25,1478.0,3649.75,23086.0
CS034515000173,10074,548.25,1478.0,3649.75,23086.0
CS022515000065,12903,548.25,1478.0,3649.75,23086.0
CS007514000094,15735,548.25,1478.0,3649.75,23086.0


---
> S-079: 商品データ（product）の各項目に対し、欠損数を確認せよ。

In [222]:
%%sql

SELECT
    SUM(CASE WHEN product_cd IS NULL THEN 1 ELSE 0 END) AS product_cd,
    SUM(CASE WHEN category_major_cd IS NULL THEN 1 ELSE 0 END) AS category_major_cd,
    SUM(CASE WHEN category_medium_cd IS NULL THEN 1 ELSE 0 END) AS category_medium_cd,
    SUM(CASE WHEN category_small_cd IS NULL THEN 1 ELSE 0 END) AS category_small_cd,
    SUM(CASE WHEN unit_price IS NULL THEN 1 ELSE 0 END) AS unit_price,
    SUM(CASE WHEN unit_cost IS NULL THEN 1 ELSE 0 END) AS unit_cost
FROM
    product
;


*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
0,0,0,0,7,7


---
> S-080: 商品データ（product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [230]:
%%sql

DROP TABLE IF EXISTS product_1;

CREATE TABLE product_1 AS (
    SELECT
        *
    FROM
        product
    WHERE
        unit_price IS NOT NULL
        OR unit_cost IS NOT NULL
);


*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
10023 rows affected.


[]

In [231]:
%%sql

SELECT
    '削除前',
    COUNT(*)
FROM
    product
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


?column?,count
削除前,10030


In [232]:
%%sql

SELECT
    '削除前',
    COUNT(*)
FROM
    product_1
;

-- 61問目-80問目：1時間44分28秒（2回目のチャレンジ）

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


?column?,count
削除前,10023


---
> S-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [145]:
%%sql

DROP TABLE IF EXISTS product_2;

CREATE TABLE product_2 AS (
    SELECT
        product_cd,
        category_major_cd,
        category_medium_cd,
        category_small_cd,
        COALESCE(unit_price, unit_avg_price) AS unit_price,
        COALESCE(unit_cost, unit_avg_cost) AS unit_cost
    FROM
        product
    CROSS JOIN (
        SELECT
            ROUND(AVG(unit_price)) AS unit_avg_price,
            ROUND(AVG(unit_cost)) AS unit_avg_cost
        FROM
            product
    ) stats_tbl
);


*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
10030 rows affected.


[]

In [146]:
%%sql

SELECT
    SUM(CASE WHEN unit_price IS NULL THEN 1 ELSE 0 END) AS unit_price,
    SUM(CASE WHEN unit_cost IS NULL THEN 1 ELSE 0 END) AS unit_cost
FROM
    product_2

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


unit_price,unit_cost
0,0


---
> S-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [148]:
%%sql

DROP TABLE IF EXISTS product_3;

CREATE TABLE product_3 AS (
    SELECT
        product_cd,
        category_major_cd,
        category_medium_cd,
        category_small_cd,
        COALESCE(unit_price, unit_med_price) AS unit_price,
        COALESCE(unit_cost, unit_med_cost) AS unit_cost
    FROM
        product
    CROSS JOIN (
        SELECT
            ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY unit_price)) AS unit_med_price,
            ROUND(PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY unit_cost)) AS unit_med_cost
        FROM
            product
    ) AS stats_tbl
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
10030 rows affected.


[]

In [152]:
%%sql

SELECT
    SUM(CASE WHEN unit_price IS NULL THEN 1 ELSE 0 END) AS unit_price,
    SUM(CASE WHEN unit_cost IS NULL THEN 1 ELSE 0 END) AS unit_cost
FROM
    product_3
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


unit_price,unit_cost
0,0


---
> S-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [155]:
%%sql

DROP TABLE IF EXISTS product_4;

CREATE TABLE product_4 AS (
    WITH stats_med_tbl AS (
        SELECT
            category_small_cd,
            ROUND(
            PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY unit_price)
            ) AS unit_med_price,
            ROUND(
            PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY unit_cost)
            ) AS unit_med_cost
        FROM
            product
        GROUP BY
            category_small_cd            
    )    
    SELECT
        p.product_cd,
        p.category_major_cd,
        p.category_medium_cd,
        p.category_small_cd,
        COALESCE(p.unit_price, s.unit_med_price) AS unit_price,
        COALESCE(p.unit_cost, s.unit_med_cost) AS unit_cost
    FROM
        product p
    INNER JOIN 
        stats_med_tbl s
    ON
        p.category_small_cd = s.category_small_cd
);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
10030 rows affected.


[]

In [156]:
%%sql

SELECT
    SUM(CASE WHEN unit_price IS NULL THEN 1 ELSE 0 END) AS unit_price,
    SUM(CASE WHEN unit_cost IS NULL THEN 1 ELSE 0 END) AS unit_cost
FROM
    product_4

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


unit_price,unit_cost
0,0


---
> S-084: 顧客データ（customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [160]:
%%sql

DROP TABLE IF EXISTS product_5;

CREATE TABLE product_5 AS (
    WITH sales_amount_2019 AS (
        SELECT
            customer_id,
            SUM(amount) AS sum_amount_2019
        FROM
            receipt
        WHERE
            sales_ymd BETWEEN 20190101 AND 20191231
        GROUP BY
            customer_id
    ), sales_amount_all AS (
        SELECT
            customer_id,
            SUM(amount) AS sum_amount_all
        FROM
            receipt
        GROUP BY
            customer_id
    )
    SELECT
        c.customer_id,
        COALESCE(sa2019.sum_amount_2019, 0) AS sales_amount_2019,
        COALESCE(saall.sum_amount_all, 0) AS sales_amount_all,
        CASE COALESCE(saall.sum_amount_all, 0)
            WHEN 0 THEN 0
            ELSE COALESCE(sa2019.sum_amount_2019, 0) * 1.0 / saall.sum_amount_all 
        END AS sales_rate
    FROM
        customer c
    LEFT JOIN
        sales_amount_2019 sa2019
    ON
        c.customer_id = sa2019.customer_id
    LEFT JOIN
        sales_amount_all saall
    ON
        c.customer_id = saall.customer_id    
)

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
21971 rows affected.


[]

In [161]:
%%sql

SELECT
    *
FROM
    product_5
WHERE
    sales_rate > 0
LIMIT
    5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


customer_id,sales_amount_2019,sales_amount_all,sales_rate
CS031415000172,2971,5088,0.58392295597484276730
CS015414000103,874,3122,0.27994875080076873799
CS011215000048,248,3444,0.07200929152148664344
CS029415000023,3767,5167,0.72904973872653377201
CS035415000029,5823,7504,0.77598614072494669510


In [163]:
%%sql

SELECT
    SUM(CASE WHEN customer_id IS NULL THEN 1 ELSE 0 END) AS customer_id,
    SUM(CASE WHEN sales_amount_2019 IS NULL THEN 1 ELSE 0 END) AS sales_amount_2019,
    SUM(CASE WHEN sales_amount_all IS NULL THEN 1 ELSE 0 END) AS sales_amount_all,
    SUM(CASE WHEN sales_rate IS NULL THEN 1 ELSE 0 END) AS sales_rate    
FROM
    product_5
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


customer_id,sales_amount_2019,sales_amount_all,sales_rate
0,0,0,0


---
> S-085: 顧客データ（customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [179]:
%%sql


DROP TABLE IF EXISTS customer_1;

CREATE TABLE customer_1 AS (
    WITH geoclde_avg AS (
        SELECT
            postal_cd,
            AVG(longitude) AS avg_longitude,
            AVG(latitude) AS avg_latitude
        FROM
            geocode
        GROUP BY
            postal_cd
    )
    SELECT
        c.*,
        g.avg_longitude,
        g.avg_latitude
    FROM
        customer c
    INNER JOIN
        geoclde_avg g
    ON
        c.postal_cd = g.postal_cd
)

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
21971 rows affected.


[]

In [181]:
%%sql

SELECT * FROM customer_1 LIMIT 3;

*  postgresql://padawan:***@db:5432/dsdojo_db
3 rows affected.


customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd,avg_longitude,avg_latitude
CS021313000114,大野 あや子,1,女性,1981-04-29,37,259-1113,神奈川県伊勢原市粟窪**********,S14021,20150905,0-00000000-0,139.3177900000000000,35.4135800000000000
CS037613000071,六角 雅彦,9,不明,1952-04-01,66,136-0076,東京都江東区南砂**********,S13037,20150414,0-00000000-0,139.8350200000000000,35.6719300000000000
CS031415000172,宇多田 貴美子,1,女性,1976-10-04,42,151-0053,東京都渋谷区代々木**********,S13031,20150529,D-20100325-C,139.6896500000000000,35.6737400000000000


---
> S-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\mbox{緯度（ラジアン）}：\phi \\
\mbox{経度（ラジアン）}：\lambda \\
\mbox{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

In [182]:
%%sql



*  postgresql://padawan:***@db:5432/dsdojo_db


---
> S-087: 顧客データ（customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [3]:
%%sql

DROP TABLE IF EXISTS customer_u;

CREATE TABLE customer_u AS (
    WITH sales_amount AS (
        SELECT
            c.customer_id,
            c.customer_name,
            c.postal_cd,
            COALESCE(SUM(r.amount),0) AS sum_amount
        FROM
            customer c
        LEFT JOIN
            receipt r
        ON c.customer_id = r.customer_id
        GROUP BY
            c.customer_id, c.customer_name, c.postal_cd 
    ), sales_ranking AS (
        SELECT
            *,
            ROW_NUMBER() OVER(
                PARTITION BY customer_name, postal_cd
                ORDER BY sum_amount DESC, customer_id) AS ranking
        FROM
            sales_amount
    )
    SELECT
        c.*
    FROM
        customer c
    INNER JOIN
        sales_ranking r
    ON
        c.customer_id = r.customer_id
        AND r.ranking = 1
);
    

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
21941 rows affected.


[]

In [5]:
%%sql

SELECT
    customer_cnt,
    customer_u_cnt,
    customer_cnt - customer_u_cnt AS diff
FROM (
    SELECT
        COUNT(*) AS customer_cnt
    FROM
        customer
) customer
CROSS JOIN (
    SELECT
        COUNT(*) AS customer_u_cnt
    FROM
        customer_u
) customer_u
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


customer_cnt,customer_u_cnt,diff
21971,21941,30


---
> S-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [10]:
%%sql

DROP TABLE IF EXISTS customer_n;

CREATE TABLE customer_n AS (
    SELECT
        c.*,
        u.customer_id AS integration_id
    FROM
        customer c
    INNER JOIN
        customer_u u
    ON c.customer_name = u.customer_name
        AND c.postal_cd = u.postal_cd
)

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
21971 rows affected.


[]

---
> S-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [16]:
%%sql

SELECT SETSEED(0.1);

CREATE TEMP TABLE IF NOT EXISTS sales_customer AS (
    SELECT
        c.customer_id,
        ROW_NUMBER() OVER(ORDER BY RANDOM()) AS row
    FROM
        customer c
    INNER JOIN
        receipt r
    ON
        c.customer_id = r.customer_id
    GROUP BY
        c.customer_id
    HAVING
        SUM(amount) > 0
);

DROP TABLE IF EXISTS customer_train;

CREATE TABLE customer_train AS (
    SELECT
        c.*
    FROM
        sales_customer s
    INNER JOIN
        customer c
    ON
        s.customer_id = c.customer_id
    WHERE
        s.row <= (
            SELECT
                COUNT(*)
            FROM
                sales_customer
        ) * 0.8
);

DROP TABLE IF EXISTS customer_test;

CREATE TABLE customer_test AS (
    SELECT
        c.*
    FROM
        sales_customer s
    INNER JOIN
        customer c
    ON
        s.customer_id = c.customer_id
    EXCEPT
        SELECT
            *
        FROM
            customer_train
);





*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.
Done.
Done.
6644 rows affected.
Done.
1662 rows affected.


[]

In [21]:
%%sql

SELECT
    train_cnt * 1.0 / all_cnt AS ratio_of_train_data,
    test_cnt * 1.0 / all_cnt AS ratio_of_test_data
FROM
    (SELECT COUNT(*) AS all_cnt FROM sales_customer) all_data
CROSS JOIN
    (SELECT COUNT(*) AS train_cnt FROM customer_train) train_data
CROSS JOIN
    (SELECT COUNT(*) AS test_cnt FROM customer_test) test_data
;

*  postgresql://padawan:***@db:5432/dsdojo_db
1 rows affected.


ratio_of_train_data,ratio_of_test_data
0.79990368408379484710,0.20009631591620515290


---
> S-090: レシート明細データ（receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [ ]:
%%sql



---
> S-091: 顧客データ（customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

In [ ]:
%%sql


---
> S-092: 顧客データ（customer）の性別について、第三正規形へと正規化せよ。

In [ ]:
%%sql


---
> S-093: 商品データ（product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [ ]:
%%sql


---
> S-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること（COPYコマンドの権限は付与済み）。
> 
> |出力先|
> |:--:|
> |/tmp/data|
> 
> ※"/tmp/data"を指定することでJupyterの"/work/data"と共有されるようになっている。

In [ ]:
%%sql

COPY product_full TO '/tmp/data/S_product_full_UTF_8_header.csv'
WITH CSV HEADER ENCODING 'UTF-8';

---
> S-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> PostgreSQLではShift_JISを指定することでCP932相当となる。ファイル出力先のパスは以下のようにすること（COPYコマンドの権限は付与済み）。
> 
> |出力先|
> |:--:|
> |/tmp/data|
> 
> ※"/tmp/data"を指定することでJupyterの"/work/data"と共有されるようになっている。

In [ ]:
%%sql

COPY product_full TO 'tmp/data/S_product_full_SJIS_header.csv'
WITH CSV HEADER ENCODING 'SJIS';

---
> S-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること（COPYコマンドの権限は付与済み）。
> 
> |出力先|
> |:--:|
> |/tmp/data|
> 
> ※"/tmp/data"を指定することでJupyterの"/work/data"と共有されるようになっている。

In [ ]:
%%sql

COPY product_full TO '/tmp/data/S_product_full_UTF-8_noh.csv'
WITH CSV ENCODING 'UTF-8'

---
> S-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

In [ ]:
%%sql


---
> S-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

In [ ]:
%%sql


---
> S-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること（COPYコマンドの権限は付与済み）。
> 
> |出力先|
> |:--:|
> |/tmp/data|
> 
> ※"/tmp/data"を指定することでJupyterの"/work/data"と共有されるようになっている。


In [ ]:
%%sql


---
> S-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

In [ ]:
%%sql


# これで１００本終わりです。おつかれさまでした！